In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize, imsave
import pdb
import glob
import os
import sys

import model 
    
flags = tf.flags
flags.DEFINE_string("models_path",'vgg16_weights.npz', "")
flags.DEFINE_string("content_image_path",'images/tubingen.jpg', "")
flags.DEFINE_string("style_image_path",'images/starry-night.jpg', "")
flags.DEFINE_float("content_loss_weight",0.999, "")
flags.DEFINE_float("style_loss_weight",1e-3, "")
flags.DEFINE_integer("max_steps",100, "")
flags.DEFINE_float("learning_rate",0.1, "")
FLAGS = flags.FLAGS
print('Imported')

In [ ]:
#################################################################################
'''
VGG16 is a pretrained model which has been trained on 1.2million images
Given below is the architecture of VGG16
conv referes to convolutional layers 
fc refers to fully connected layers
'''
#################################################################################
vgg_layers = ['conv1_1', 'conv1_2', 'pool1', 
              'conv2_1', 'conv2_2', 'pool2', 
              'conv3_1', 'conv3_2', 'conv3_3', 'pool3',
              'conv4_1', 'conv4_2', 'conv4_3', 'pool4',
              'conv5_1', 'conv5_2', 'conv5_3', 'pool5',
              'fc6', 'fc7', 'fc8']


In [ ]:
#################################################################################
'''
Function which computes the gram matrix given a vector tensor 
'''
#################################################################################
def gram_matrix(feature_maps):
  """Computes the Gram matrix for a set of feature maps."""
  batch_size, height, width, channels = tf.unstack(tf.shape(feature_maps))
  denominator = tf.to_float(height * width)
  feature_maps = tf.reshape(
      feature_maps, tf.stack([batch_size, height * width, channels]))
  matrix = tf.matmul(feature_maps, feature_maps, adjoint_a=True)
  return matrix / denominator

In [ ]:
###########################################################################
'''
Read the content and style images 
The images are of shape: 224,224,3 #RGB channels 
'''
#################################################################################
    
# Open content image and resize it 
content_image = imread(FLAGS.content_image_path)[:,:,:3]
content_image = np.expand_dims(imresize(content_image, (224, 224)), 0)
    
# Open style image and resize it 
style_image = imread(FLAGS.style_image_path)[:,:,:3]
style_image = np.expand_dims(imresize(style_image, (224, 224)), 0)

In [ ]:
#################################################################################
'''
Create Graph first by defining placeholders, ops, loss functions and optimizers

'''
#################################################################################
tf.reset_default_graph()


#################################################################################
'''Placeholders'''
#################################################################################
content_tensor = '''Fill your code here''' #shape=[1,224,224,3] 
style_tensor = '''Fill your code here''' #shape=[1,224,224,3] 
    
                   
# Define the output tensor as a Variable - so we can train it
# The output tensor should be initialized with the content image
#################################################################################
'''Output Tensor - initialize with content image - which we will train'''
#################################################################################    
output_tensor = tf.Variable('''Fill your code here''') #shape=[1,224,224,3] 
    

#################################################################################
'''
The class object for the VGG16 neural network
Propagate the content_tensor, style_tensor and the output tensor through this model
'''
#################################################################################    
vgg = model.vgg16()
        
# Forward pass the content tensors
with tf.variable_scope('vgg') as scope:
    content_end_points = vgg.forward(content_tensor)
    # content_end_points are a dictionary of (key, value) pairs with 
    # keys=layer_names and values=activationsis a dictionary of 
        
# Forward pass the style tensors - make sure to reuse the variables - 
# otherwise a new set of variables for vgg16 will be defined
with tf.variable_scope('vgg') as scope:
    scope.reuse_variables()
    style_end_points = '''Fill your code here'''
        
# Forward pass the output tensors - make sure to reuse the variables - 
# otherwise a new set of variables for vgg16 will be defined
with tf.variable_scope('vgg') as scope:
    scope.reuse_variables()
    output_end_points = '''Fill your code here'''
        

#################################################################################    
''' Loss computation '''
#################################################################################    
content_layers = ['conv1_2']
style_layers = ['conv1_2', 'conv2_2', 'conv3_2', 'conv4_2', 'conv5_2']
    
    
# Content loss
content_loss = 0
for layer in content_layers:
    # MSE of the activations 
    mse = tf.losses.mean_squared_error(content_end_points[layer], output_end_points[layer])
    # Add it to the content_loss 
    content_loss += mse
        
# Style loss
style_loss = 0
for layer in style_layers:
    # Compute gram matrix of the activations for style and output
    gram_matrix_style = gram_matrix(style_end_points[layer])
    gram_matrix_output = gram_matrix(output_end_points[layer])
        
    # MSE of the gram matrix
    '''Fill your code here'''
        
    # Add it to the style_loss 
    '''Fill your code here'''
        
# Combine the content and style losses - one can also weigh them differently
total_loss = '''Fill your code here'''    
    
#################################################################################    
'''Optimizers '''
#################################################################################    
# Define optimizers and pass only output_tensor as variable to be trained
learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate)
optimizer_step = optimizer.minimize(total_loss, var_list=[output_tensor])

In [ ]:
#################################################################################    
'''
Create a TF session and execute the graph
'''
#################################################################################    

sess = tf.Session()

'''Initialize and loading the weights'''
#Initialize the variables and load the weights for VGG
sess.run(tf.global_variables_initializer())
vgg.load_weights(FLAGS.models_path,sess)
    
    
#################################################################################    
'''
Execute the graph several times to get the desired output
'''   
#################################################################################    
for i in range(FLAGS.max_steps):
    # sess.run() should consist of 'fetch' and 'feed' 
    # 'fetch' are the list of variables to be returned
    # 'feed' is the dictionary of values assigned to respective placeholders
    fetch = [optimizer_step, total_loss, content_loss, style_loss, output_tensor]
    feed = '''Fill your code here'''
    # Define the session.run() with fetch and feed parameters
    _, loss, c_loss, s_loss, output_image = '''Fill your code here'''
    print('Iteration : %4d, total_loss : %g, content_loss : %g, style_loss : %g' % (i,loss, c_loss, s_loss))
    if i%10==0:
        plt.imshow(np.hstack([content_image[0],output_image[0],style_image[0] ])/output_image.max()); plt.show()
    print('Completed')
